In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon May  1 18:23:43 2017

@author: sergej
"""
# for one year

from bokeh.io import curdoc
from bokeh.models import ColumnDataSource, CategoricalColorMapper, Slider, Select
from bokeh.plotting import figure, output_notebook, show
from bokeh.palettes import Spectral6
from bokeh.layouts import widgetbox, row
import pandas as pd
import numpy as np 


In [2]:
MPI_WorldBank_DPI_cont=pd.read_csv('/Users/sergej/Google Drive/QASS_Master/Thesis_QASS/Data/MPI_WorldBank_DPI_cont.csv')
MPI_WorldBank_DPI_cont=MPI_WorldBank_DPI_cont.reset_index().rename(columns={'NY_GDP_PCAP_KD':'gdppc'})
data=MPI_WorldBank_DPI_cont
data.set_index(['year'])
data.columns=[x.strip().replace(' ', '_') for x in data.columns]
data=data.drop('index_x',axis=1)
data=data.drop('Unnamed:_0',axis=1)

# Create a HoverTool: hover
# Make a list of the unique values from the region column: regions_list


In [3]:
regions_list = data.continent.unique().tolist()

# Import CategoricalColorMapper from bokeh.models and the Spectral6 palette from bokeh.palettes

xmin, xmax = min(data.gdppc), max(data.gdppc)

# Save the minimum and maximum values of the life expectancy column: ymin, ymax
ymin, ymax = min(data.MPI_E), max(data.MPI_E)
output_notebook()
data=data.set_index('year',drop=False)

source = ColumnDataSource( data = {
 'x'       : data.loc[1996].gdppc,
 'y'       : data.loc[1996].MPI_E,
 'country' : data.loc[1996].Country_Code,
 'region' : data.loc[1996].continent,
 'pop'     : (data.loc[1996].SP_POP_TOTL / 20000000) + 2,
 'unemployment'  : data.loc[1996].SL_UEM_TOTL_ZS,
 'Immigration_Integration':data.loc[1996].MPI_I,
 'Immigration_Stay':data.loc[1996].MPI_S,
 'Immigration_Overall':data.loc[1996].MPI,
 'Age_dependency_ratio':data.loc[1996].SP_POP_DPND                   
  })
    

# Create layout: layout
#layout = row(widgetbox(slider),plot)

# Add layout to current document
#curdoc().add_root(layout)

#figure: plot
plot = figure(title='Immigration Data', plot_height=400, plot_width=700,
              x_range=(xmin, xmax), y_range=(ymin, ymax))

# Make a color mapper: color_mapper
color_mapper = CategoricalColorMapper(factors=regions_list, palette=Spectral6)

# Add the color mapper to the circle glyph
plot.circle(x='x', y='y', fill_alpha=0.8, source=source,
            color=dict(field='region', transform=color_mapper), legend='region')

# Set the legend.location attribute of the plot to 'top_right'
plot.legend.location = 'top_right'

# Add the plot to the current document and add the title
curdoc().add_root(plot)


# Import HoverTool from bokeh.models
from bokeh.models import HoverTool
# Save the minimum and maximum values of the fertility column: xmin, xmax

# Create a HoverTool: hover
# Create the f
hover = HoverTool(tooltips=[('Country', '@country'),('Region','@region'),('GDP_p_c','@gdppc')])

# Add the HoverTool to the plot
plot.add_tools(hover)
# Create layout: layout

#layout = row(widgetbox(slider),plot)

# Add layout to current document
#curdoc().add_root(layout)

#Adding dropdowns to the app
# Define the callback: update_plot


Loading BokehJS ...

In [4]:
def update_plot(attr, old, new):
    # Read the current value off the slider and 2 dropdowns: yr, x, y
    yr = slider.value
    x = x_select.value
    y = y_select.value
    # Label axes of plot
    plot.xaxis.axis_label = x
    plot.yaxis.axis_label = y
    # Set new_data
    new_data = {
        'x'       : data.loc[yr][x],
        'y'       : data.loc[yr][y],
        'country' : data.loc[yr].Country_Code,
        'region' : data.loc[yr].continent,
        'pop'     : (data.loc[yr].SP_POP_TOTL / 20000000) + 2,
        'unemployment'  : data.loc[yr].SL_UEM_TOTL_ZS,
        'GDP_p_c': data.loc[yr].gdppc,
        'Immigration_Entry':data.loc[yr].MPI_E,
        'Immigration_Integration':data.loc[yr].MPI_I,
        'Immigration_Stay':data.loc[yr].MPI_S,
        'Immigration_Overall':data.loc[yr].MPI,
    'Age_dependency_ratio':data.loc[yr].SP_POP_DPND                   
    }
    # Assign new_data to source.data
    source.data = new_data

    # Set the range of all axes
    plot.x_range.start = min(data[x])
    plot.x_range.end = max(data[x])
    plot.y_range.start = min(data[y])
    plot.y_range.end = max(data[y])

    # Add title to plot
    plot.title.text = 'Immigration Policy data for %d' % yr



In [5]:
# Create a dropdown slider widget: slider
slider = Slider(start=1993, end=2014, step=1, value=1993, title='Year')

# Attach the callback to the 'value' property of slider
slider.on_change('value', update_plot)

# Create a dropdown Select widget for the x data: x_select
x_select = Select(
    options=[        
        'pop'     ,
        'unemployment'  ,
        'GDP_p_c',
    'Age_dependency_ratio'],
    value='MPI_I',
    title='x-axis data'
)

# Attach the update_plot callback to the 'value' property of x_select
x_select.on_change('value', update_plot)

# Create a dropdown Select widget for the y data: y_select
y_select = Select(
    options=['Immigration_Entry',
        'Immigration_Integration',
        'Immigration_Stay',
        'Immigration_Overall'],
    value='Immigration_Entry',
    title='y-axis data'
)

# Attach the update_plot callback to the 'value' property of y_select
y_select.on_change('value',update_plot)

# Create layout and add to current document
layout = row(widgetbox(slider, x_select, y_select), plot)
curdoc().add_root(layout)



In [6]:
output_notebook()
show(plot)

Loading BokehJS ...

ValueError: Out of range float values are not JSON compliant